In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import Ridge
import math
from itertools import zip_longest
import random
from feature_selection import get_elastic_importances, get_lasso_importances, get_anova_importances
from collections import defaultdict
from sklearn.feature_selection import f_classif
#to get rid of the warning
import warnings
warnings.filterwarnings('ignore')


In [9]:
#data
data = pd.read_csv('Fish3.txt', delimiter=' ')

X = data.drop('Species', axis=1)
target = data['Species']
seed = 42
random.seed(seed)
np.random.seed(seed)

X_train, X_test, target_train, target_test = train_test_split(X, target, test_size=0.3, random_state=seed)
print("X_train shape:" , target_train)

mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std


X_train shape: 1377          Bream
209           Smelt
1310          Bream
1461           Pike
59             Pike
           ...     
1131          Perch
1295          Perch
861     SilverBream
1460          Smelt
1127    SilverBream
Name: Species, Length: 1306, dtype: object


In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

classifiers = {
    'RandomForest': RandomForestClassifier(n_estimators=100),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'KNN-3': KNeighborsClassifier(n_neighbors=3),
}

results = {}

# For each classifier
for classifier_name, clf in classifiers.items():
    
    baseline_scores = cross_val_score(clf, X_train, target_train, cv=5)
    baseline_accuracy = np.mean(baseline_scores)
    most_important_feature = None
    max_decrease = 0
    for i in range(X_train.shape[1]):
        
        X_train_removed = X_train.copy()
        
        X_train_removed = np.delete(X_train_removed, i, axis=1)
        
        scores_removed = cross_val_score(clf, X_train_removed, target_train, cv=5)
        accuracy_removed = np.mean(scores_removed)
        decrease = baseline_accuracy - accuracy_removed

        # Update the most important feature and its decrease in accuracy
        if decrease > max_decrease:
            most_important_feature = i
            max_decrease = decrease

        results[(classifier_name, i)] = decrease

    # Print the most important feature for this classifier
    print(f'{classifier_name}, most important feature: {most_important_feature}, decrease in accuracy = {max_decrease}')

# Print the results
for (classifier_name, i), decrease in results.items():
    print(f'{classifier_name}, feature {i}: Decrease in accuracy = {decrease}')
#         results[(classifier_name, i)] = baseline_accuracy - accuracy_removed

# # Print the results
# for (classifier_name, i), decrease in results.items():
#     print(f'{classifier_name}, feature {i}: Decrease in accuracy = {decrease}')


RandomForest, most important feature: 4, decrease in accuracy = 0.0980170220233395
LogisticRegression, most important feature: 4, decrease in accuracy = 0.10797870784709429
KNN-3, most important feature: 4, decrease in accuracy = 0.08117048346055955
RandomForest, feature 0: Decrease in accuracy = 0.005361059928051359
RandomForest, feature 1: Decrease in accuracy = -8.774238834496906e-06
RandomForest, feature 2: Decrease in accuracy = -5.849492556442293e-06
RandomForest, feature 3: Decrease in accuracy = 0.009186628060015911
RandomForest, feature 4: Decrease in accuracy = 0.0980170220233395
RandomForest, feature 5: Decrease in accuracy = 0.031400076043403446
LogisticRegression, feature 0: Decrease in accuracy = -0.0015384165423648621
LogisticRegression, feature 1: Decrease in accuracy = 0.0038284928782426064
LogisticRegression, feature 2: Decrease in accuracy = 0.006127343452955403
LogisticRegression, feature 3: Decrease in accuracy = 0.024509373811821833
LogisticRegression, feature 4: 

In [15]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression(max_iter=1000)

selector = RFE(estimator, n_features_to_select=5, step=1)

selector = selector.fit(X_train, target_train)


ranking = selector.ranking_

for i, rank in enumerate(ranking):
    print(f"Feature {i}: Rank = {rank}")

Feature 0: Rank = 1
Feature 1: Rank = 2
Feature 2: Rank = 1
Feature 3: Rank = 1
Feature 4: Rank = 1
Feature 5: Rank = 1
